# Assignment - MMB
*Alexander Laloi Dybdahl, Valentin Vuillon, Alexia Stéphanie Liviana Paratte*

In [2]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, exp, Derive
import scipy.stats as st


### Loading data

In [3]:
df = pd.read_csv("lpmc07.dat", delimiter='\t')

## Tasks

### Model 0

Model 0 includes a general cost parameter and alternative-specific time parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} + \beta_{\text{TIME}} \cdot \text{dur\_walking} + \epsilon_{\text{walk}} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} + \beta_{\text{TIME}} \cdot \text{dur\_cycling} + \epsilon_{\text{cycle}} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC\_PT} + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME}} \cdot \text{dur\_pt\_total} + \epsilon_{\text{pt}} $$

- **Driving**:  
  $$ U_{\text{drive}} = \text{ASC\_DRIVE} + \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME}} \cdot \text{dur\_driving} + \epsilon_{\text{drive}} $$

where:
- $ \beta_{\text{COST}} $ is the coefficient for travel cost.
- $ \beta_{\text{TIME}} $ is the coefficient for travel time.
- $ \text{cost}_j $ is the travel cost for mode $ j $.
- $ \text{dur}_j $ is the travel time for mode $ j $.
- $ \epsilon_j $ is the error term, representing unobserved factors affecting the utility of mode $ j $.

The probability $ P_j $ of choosing mode $ j $ is given by the softmax function:

$$ P_j = \frac{\exp(U_j)}{\sum_{k=1}^{J} \exp(U_k)} $$


In [4]:
# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']
df['car_available'] = (df['car_ownership'] > 0).astype(int)
median_distance = df['distance'].median()
df['short_distance'] = (df['distance'] < median_distance).astype(int)
df['young'] = (df['age'] < 30).astype(int)

# Create a Biogeme database
database = db.Database('LPMC', df)
globals().update(database.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)

BETA_COST = Beta('BETA_COST', 0, None, None, 0)
BETA_TIME = Beta('BETA_TIME', 0, None, None, 0)

# Define utility functions using Biogeme expressions
V1 = ASC_WALK + BETA_TIME * dur_walking
V2 = ASC_BIKE + BETA_TIME * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + BETA_TIME * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_0 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_0 = bio.BIOGEME(database, logprob_0)
biogeme_0.modelName = 'Model_0'
biogeme_0.generateHtml = False  # Disable HTML file generation
biogeme_0.generatePickle = False  # Disable PICKLE file generation
biogeme_0.save_iterations = False  # Disable ITER file generation
results_model_0 = biogeme_0.estimate()

# Output
print(results_model_0.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE  -2.569395      0.090262   -28.466008           0.0
ASC_PT     0.766416      0.047360    16.182859           0.0
ASC_WALK   1.256089      0.076712    16.374053           0.0
BETA_COST -0.173019      0.014562   -11.881542           0.0
BETA_TIME -5.326760      0.189549   -28.102298           0.0


In [5]:
# Retrieve the general statistics from the results
general_stats_model_0 = results_model_0.getGeneralStatistics()
print(results_model_0.printGeneralStatistics())


Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4642.324
Likelihood ratio test for the init. model:	4578.295
Rho-square for the init. model:	0.33
Rho-square-bar for the init. model:	0.33
Akaike Information Criterion:	9294.648
Bayesian Information Criterion:	9327.234
Final gradient norm:	7.3390E-04
Nbr of threads:	8



#### Critival value for t-test

Using a siginificance level $\alpha = 0.05$, we get:

In [6]:
alpha = 0.05
t = st.distributions.t.ppf(1 - alpha/2, 4999)
print(t)

1.9604386466615242


### Model 1

Model 1 includes alternative-specific cost parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking} + \epsilon_{\text{walk}} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling} + \epsilon_{\text{cycle}} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC\_PT} + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total} + \epsilon_{\text{pt}} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving} + \epsilon_{\text{drive}} $$

Where:
- $ \text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT} $ are the alternative specific constants for walking, cycling, and public transport, respectively.
- $ \beta_{\text{COST}} $ is the common cost coefficient for all transportation modes.
- $ \beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}} $ are the time coefficients for walking, cycling, public transport, and driving, respectively.
- $ \text{cost\_transit}, \text{cost\_driving\_total} $ are the costs associated with public transport and driving.
- $ \text{dur\_walking}, \text{dur\_cycling}, \text{dur\_pt\_total}, \text{dur\_driving} $ are the travel durations for each mode.


In [7]:

# Define additional parameters for the cost for each mode
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# Define utility functions using Biogeme expressions with alternative-specific cost coefficients
V1 = ASC_WALK + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_1 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_1 = bio.BIOGEME(database, logprob_1)
biogeme_1.modelName = 'Model_1'
biogeme_1.generateHtml = False  # Disable HTML file generation
biogeme_1.generatePickle = False  # Disable PICKLE file generation
biogeme_1.save_iterations = False  # Disable ITER file generation
results_model_1 = biogeme_1.estimate()

# Output
print(results_model_1.getEstimatedParameters())

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -2.330094      0.157067   -14.835063      0.000000
ASC_PT          -0.321680      0.068372    -4.704882      0.000003
ASC_WALK         2.078822      0.134683    15.434922      0.000000
BETA_COST       -0.163540      0.016154   -10.123737      0.000000
BETA_TIME_BIKE  -6.714893      0.589110   -11.398371      0.000000
BETA_TIME_DRIVE -5.841121      0.364543   -16.023119      0.000000
BETA_TIME_PT    -3.291291      0.235273   -13.989233      0.000000
BETA_TIME_WALK  -8.487469      0.409812   -20.710663      0.000000


In [8]:
# Retrieve the general statistics from the results
general_stats_model_1 = results_model_1.getGeneralStatistics()
print(results_model_1.printGeneralStatistics())


Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4329.182
Likelihood ratio test for the init. model:	5204.58
Rho-square for the init. model:	0.375
Rho-square-bar for the init. model:	0.374
Akaike Information Criterion:	8674.363
Bayesian Information Criterion:	8726.501
Final gradient norm:	2.0146E-03
Nbr of threads:	8



**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{pt}}, \text{and} \text{ASC}_{\text{walk}} $ are statistically significant with near-zero p-values. The constants indicate a baseline aversion to cycling ($ \text{ASC}_{\text{bike}} < 0 $) and a preference for walking, driving, and public transport ($ \text{ASC}_{\text{walk}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}} > 0 $).

**Alternative-Specific Cost Coefficients:**

- $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ are zero, indicating no significant impact of costs on biking and walking utility.
- $ \beta_{\text{cost\_drive}} $ is negative and significant, suggesting higher driving costs reduce its utility.
- $ \beta_{\text{cost\_pt}} $ is positive and significant, an unexpected result implying higher public transport costs might correlate with higher utility, potentially reflecting unmodeled factors like income or service quality.

**Time Coefficient ($ \beta_{\text{time}} $):**

- Remains negative and significant, indicating longer travel times decrease the utility of a mode.


### Comparing $\text{Model 1}$ and Model 0

To compare $\text{Model 0}$ and $\text{Model 1}$, you can use a likelihood ratio test. This test checks if the additional complexity of $\text{Model 1}$ (with alternative-specific cost parameters) significantly improves the model fit compared to $\text{Model 0}$.

- **Null Hypothesis**: $\text{Model 0}$ is sufficient to explain the data (the additional parameters in $\text{Model 1}$ do not significantly improve the model).

- **Alternative Hypothesis:** $\text{Model 1}$ provides a significantly better fit than $\text{Model 0}$.

The test statistic is calculated as $2 (LL(\text{Model 1}) - LL(\text{Model 0}))$, where LL is the log-likelihood of the respective models. This statistic follows a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models.

Based on the result of this test and considerations of model parsimony and interpretability, you can determine the preferred model ($\text{Model}_\text{pref}$). Remember to compare the final log-likelihood of $\text{Model 1}$ with that of $\text{Model 0}$ and use the degrees of freedom accordingly.

In [9]:
LR_test = 2 * (results_model_1.data.logLike - results_model_0.data.logLike)
print("Log likelihood ratio:", LR_test)
p_val = st.chi2.sf(LR_test, 3)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 3)
print("Critical value:", x_qhi)

# Get general statistics for Model 1
general_stats_model_1 = results_model_1.getGeneralStatistics()
print(results_model_1.printGeneralStatistics())

Log likelihood ratio: 626.285269888871
p value: 2.0178995664426872e-135
Critical value: 0.35184631774927144
Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4329.182
Likelihood ratio test for the init. model:	5204.58
Rho-square for the init. model:	0.375
Rho-square-bar for the init. model:	0.374
Akaike Information Criterion:	8674.363
Bayesian Information Criterion:	8726.501
Final gradient norm:	2.0146E-03
Nbr of threads:	8



#### Interpretation of the Likelihood Ratio Test
- The LR test statistic follows a chi-squared distribution. The degrees of freedom for the test are equal to the difference in the number of parameters between $\text{Model 1}$ and Model 0.

- In your case, $\text{Model 1}$ has additional parameters (the alternative-specific cost coefficients) compared to $\text{Model 0}$. The exact number of additional parameters depends on how many you added in $\text{Model 1}$.

#### Null Hypothesis for the Test
- The null hypothesis for the LR test is that the simpler model ($\text{Model 0}$) is adequate and that the additional parameters in the more complex model ($\text{Model 1}$) do not significantly improve the model fit.

#### Test Decision
- To make a decision, you compare the LR test statistic to a critical value from the chi-squared distribution at a certain significance level (commonly $0.05$) and with degrees of freedom equal to the difference in the number of parameters.
- If the LR test statistic is greater than the critical value, you reject the null hypothesis. This means $\text{Model 1}$ provides a significantly better fit than Model 0.

#### Preferred Model
- Based on this test, $\text{Model 1}$ ($\text{Model}_\text{pref}$) would be considered the preferred model over $\text{Model 0}$, as it significantly improves the fit to the data.
- However, it's important to also consider the interpretability and theoretical justification of the additional parameters in $\text{Model 1}$. Sometimes a more complex model is not preferable if it does not add meaningful explanatory power or if it makes the model less interpretable.

### Model 2

Model 2 introduces interactions with the socio-economic characteristic variable $\text{young}$, building upon the specifications from $\text{Model\_pref}$. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} \cdot \text{young} + \text{ASC\_WALK\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{TIME\_WALK}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} \cdot \text{young} + \text{ASC\_BIKE\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{TIME\_BIKE}} \cdot \text{dur\_cycling} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC\_PT} \cdot \text{young} + \text{ASC\_PT\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \text{dur\_driving} $$

Where:
- $\text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT}$ are the alternative specific constants.
- $\text{ASC\_WALK\_NOTYOUNG}, \text{ASC\_BIKE\_NOTYOUNG}, \text{ASC\_PT\_NOTYOUNG}$ are the constants for when $\text{young}$ is 0.
- $\beta_{\text{COST}}$ is the cost coefficient applicable to public transport and driving.
- $\beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}}$ are the time coefficients for each mode.
- $\text{young}$ is a binary variable indicating car availability.





#### Interaction with ASC

In [10]:

# Define additional parameters for the utility functions
ASC_WALK_YOUNG = Beta('ASC_WALK_YOUNG', 0, None, None, 0)
ASC_BIKE_YOUNG = Beta('ASC_BIKE_YOUNG', 0, None, None, 0)
ASC_PT_YOUNG = Beta('ASC_PT_YOUNG', 0, None, None, 0)
ASC_WALK_NOTYOUNG = Beta('ASC_WALK_NOTYOUNG', 0, None, None, 0)
ASC_BIKE_NOTYOUNG = Beta('ASC_BIKE_NOTYOUNG', 0, None, None, 0)
ASC_PT_NOTYOUNG = Beta('ASC_PT_NOTYOUNG', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK_YOUNG * young + ASC_WALK_NOTYOUNG * (1 - young) + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE_YOUNG * young + ASC_BIKE_NOTYOUNG * (1 - young) + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT_YOUNG * young + ASC_PT_NOTYOUNG * (1 - young) + BETA_COST * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_2 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_2 = bio.BIOGEME(database, logprob_2)
biogeme_2.modelName = 'Model_2'
biogeme_2.generateHtml = False  # Disable HTML file generation
biogeme_2.generatePickle = False  # Disable PICKLE file generation
biogeme_2.save_iterations = False  # Disable ITER file generation
results_model_2_0 = biogeme_2.estimate()

# Output
print(results_model_2_0.getEstimatedParameters())
# Get general statistics
print( results_model_2_0.printGeneralStatistics())


Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                      Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE_NOTYOUNG -2.436392      0.168914   -14.423819  0.000000e+00
ASC_BIKE_YOUNG    -2.103990      0.199249   -10.559616  0.000000e+00
ASC_PT_NOTYOUNG   -0.538214      0.074436    -7.230518  4.811707e-13
ASC_PT_YOUNG       0.089879      0.085230     1.054536  2.916376e-01
ASC_WALK_NOTYOUNG  1.910612      0.135148    14.137193  0.000000e+00
ASC_WALK_YOUNG     2.485186      0.162019    15.338891  0.000000e+00
BETA_COST         -0.156258      0.015741    -9.926732  0.000000e+00
BETA_TIME_BIKE    -6.791816      0.592522   -11.462563  0.000000e+00
BETA_TIME_DRIVE   -5.988674      0.368419   -16.255079  0.000000e+00
BETA_TIME_PT      -3.365843      0.236718   -14.218809  0.000000e+00
BETA_TIME_WALK    -8.575972      0.417399   -20.546204  0.000000e+00
Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4289.032
Likelihood ratio test for the i

#### Interaction with the time

In [11]:

# Define additional parameters for the utility functions
BETA_TIME_AGE_WALK = Beta('BETA_TIME_AGE_WALK', 0, None, None, 0)
BETA_TIME_AGE_BIKE = Beta('BETA_TIME_AGE_BIKE', 0, None, None, 0)
BETA_TIME_AGE_PT = Beta('BETA_TIME_AGE_PT', 0, None, None, 0)
BETA_TIME_AGE_DRIVE = Beta('BETA_TIME_AGE_DRIVE', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK + (BETA_TIME_WALK + BETA_TIME_AGE_WALK * age) * dur_walking
V2 = ASC_BIKE + (BETA_TIME_BIKE + BETA_TIME_AGE_BIKE * age) * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + (BETA_TIME_PT + BETA_TIME_AGE_PT * age) * dur_pt_total
V4 = BETA_COST * cost_driving_total + (BETA_TIME_DRIVE + BETA_TIME_AGE_DRIVE * age) * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_2 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_2 = bio.BIOGEME(database, logprob_2)
biogeme_2.modelName = 'Model_2'
biogeme_2.generateHtml = False  # Disable HTML file generation
biogeme_2.generatePickle = False  # Disable PICKLE file generation
biogeme_2.save_iterations = False  # Disable ITER file generation
results_model_2_1 = biogeme_2.estimate()

# Output
print(results_model_2_1.getEstimatedParameters())
# Get general statistics
print( results_model_2_1.printGeneralStatistics())


Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                        Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE            -2.315993      0.157465   -14.708003  0.000000e+00
ASC_PT              -0.320809      0.068653    -4.672916  2.969526e-06
ASC_WALK             2.107021      0.136060    15.485940  0.000000e+00
BETA_COST           -0.170526      0.016678   -10.224526  0.000000e+00
BETA_TIME_AGE_BIKE  -0.045730      0.021856    -2.092385  3.640405e-02
BETA_TIME_AGE_DRIVE -0.009309      0.020734    -0.448947  6.534702e-01
BETA_TIME_AGE_PT    -0.028741      0.011307    -2.541934  1.102409e-02
BETA_TIME_AGE_WALK  -0.032609      0.009064    -3.597502  3.212880e-04
BETA_TIME_BIKE      -5.000172      0.945338    -5.289297  1.227876e-07
BETA_TIME_DRIVE     -5.454137      0.849120    -6.423279  1.333700e-10
BETA_TIME_PT        -2.153586      0.482808    -4.460546  8.175118e-06
BETA_TIME_WALK      -7.324360      0.503450   -14.548332  0.000000e+00
Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0

**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}}, \text{and } \text{ASC}_{\text{walk}} $ are all statistically significant with p-values close to zero.

**Cost Coefficients:**

- $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ are zero, implying that costs do not significantly influence the utility of biking and walking.
- $ \beta_{\text{cost\_drive}} $ is negative and significant, meaning higher driving costs reduce its utility.
- $ \beta_{\text{cost\_pt}} $ is positive but not statistically significant, indicating that cost variations in public transport do not substantially affect its utility.

**Interaction Terms:**

- $ \beta_{\text{drive\_carown}} $ is positive and significant, highlighting that car ownership notably increases the utility of driving.
- $ \beta_{\text{cost\_drive\_carown}} $, while positive, is not statistically significant, suggesting that the interaction effect of driving costs and car ownership on driving utility is indeterminate in this model.

**Time Coefficient:**

- $ \beta_{\text{time}} $ remains negative and significant, reinforcing that longer travel times reduce the utility of all modes.

**Interpretation and Implications:**

- The negative $ \text{ASC}_{\text{drive}} $ reflects a shift in baseline driving preference when considering car ownership, underscored by the significant positive interaction with car ownership.
- The positive and significant $ \beta_{\text{drive\_carown}} $ aligns with the intuitive expectation that car ownership increases the utility of driving.
- The indistinct impact of $ \beta_{\text{cost\_drive\_carown}} $ suggests that car owners' sensitivity to driving costs may not differ notably from non-owners in this dataset.
- The zero coefficients for $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ continue to indicate that cost is not a pivotal factor in the choice to walk or cycle.


### Comparing Model 2 and $\text{Model 1}$

**Model Comparison ($\text{Model}_\text{pref}$ vs. $\text{Model 2}$):**
To compare $\text{Model 2}$ with $\text{Model}_\text{pref}$, you can use a likelihood ratio test:

- **Null Hypothesis:** $\text{Model}_\text{pref}$ is sufficient, and the additional interaction terms in $\text{Model 2}$ do not significantly improve the model.
- **Alternative Hypothesis:** $\text{Model 2}$ provides a significantly better fit than $\text{Model}_\text{pref}$.

Calculate the LR test statistic and compare it to a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models. The decision on the preferred model should consider both statistical significance and the interpretability of the model.

In [12]:
LR_test = 2 * (results_model_2_0.data.logLike - results_model_1.data.logLike)
print("Log likelihood ratio test:", LR_test)
p_val = st.chi2.sf(LR_test, 3)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 3)
print("Critical value:", x_qhi)


# Get general statistics for Model 2
general_stats_model_2_0 = results_model_2_0.getGeneralStatistics()
print(results_model_2_0.printGeneralStatistics())

Log likelihood ratio test: 80.29882788641771
p value: 2.6481136977156633e-17
Critical value: 0.35184631774927144
Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4289.032
Likelihood ratio test for the init. model:	5284.879
Rho-square for the init. model:	0.381
Rho-square-bar for the init. model:	0.38
Akaike Information Criterion:	8600.064
Bayesian Information Criterion:	8671.753
Final gradient norm:	3.3781E-03
Nbr of threads:	8



In [13]:
LR_test = 2 * (results_model_2_1.data.logLike - results_model_1.data.logLike)
print("Log likelihood ratio test:", LR_test)
p_val = st.chi2.sf(LR_test, 4)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 4)
print("Critical value:", x_qhi)

# Get general statistics for Model 2
general_stats_model_2_1 = results_model_2_1.getGeneralStatistics()
print(results_model_2_1.printGeneralStatistics())

Log likelihood ratio test: 63.406664882097175
p value: 5.571981579998191e-13
Critical value: 0.7107230213973239
Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4297.478
Likelihood ratio test for the init. model:	5267.987
Rho-square for the init. model:	0.38
Rho-square-bar for the init. model:	0.378
Akaike Information Criterion:	8618.956
Bayesian Information Criterion:	8697.163
Final gradient norm:	9.1581E-03
Nbr of threads:	8




- Calculated LR test statistic:
- **Interpretation**:
  - The high value of the LR test statistic suggests that $\text{Model 2}$ provides a significantly better fit to the data compared to $\text{Model 1}$.
- **Test Decision**:
  - With a large LR statistic, the null hypothesis (that $\text{Model 1}$ is sufficient) is likely rejected, indicating a preference for $\text{Model 2}$.
- **Conclusion**:
  - $\text{Model 2}$, with its additional parameters and interactions, is the preferred model over $\text{Model 1}$, given its significantly better fit to the data.


### Model 3

Model 3 integrates a non-linear transformation (specifically, a logarithmic transformation) for travel durations in its utility functions. The utility functions are now defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC\_WALK} \cdot \text{young} + \text{ASC\_WALK\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{TIME\_WALK}} \cdot \log(\text{dur\_walking} + 1) $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC\_BIKE} \cdot \text{young} + \text{ASC\_BIKE\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{TIME\_BIKE}} \cdot \log(\text{dur\_cycling} + 1) $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC\_PT} \cdot \text{young} + \text{ASC\_PT\_NOTYOUNG} \cdot (1 - \text{young}) + \beta_{\text{COST}} \cdot \text{cost\_transit} + \beta_{\text{TIME\_PT}} \cdot \log(\text{dur\_pt\_total} + 1) $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{COST}} \cdot \text{cost\_driving\_total} + \beta_{\text{TIME\_DRIVE}} \cdot \log(\text{dur\_driving} + 1) $$

Where:
- $\text{ASC\_WALK}, \text{ASC\_BIKE}, \text{ASC\_PT}$ are the alternative specific constants.
- $\text{ASC\_WALK\_NOTYOUNG}, \text{ASC\_BIKE\_NOTYOUNG}, \text{ASC\_PT\_NOTYOUNG}$ are constants for when $\text{young}$ is 0.
- $\beta_{\text{COST}}$ is the cost coefficient applicable to public transport and driving.
- $\beta_{\text{TIME\_WALK}}, \beta_{\text{TIME\_BIKE}}, \beta_{\text{TIME\_PT}}, \beta_{\text{TIME\_DRIVE}}$ are the coefficients for the logarithmic transformation of the travel durations for each mode.
- $\text{young}$ is a binary variable indicating car availability.


In [14]:

# Define boxcox parameters
LAMBDA_BOXCOX = Beta('LAMBDA_BOXCOX', 1, -10, 10, 0)
BOXCOX_TIME_1 = models.boxcox(dur_walking, LAMBDA_BOXCOX)
BOXCOX_TIME_2 = models.boxcox(dur_cycling, LAMBDA_BOXCOX)
BOXCOX_TIME_3 = models.boxcox(dur_pt_total, LAMBDA_BOXCOX)
BOXCOX_TIME_4 = models.boxcox(dur_driving, LAMBDA_BOXCOX)

# Define utility function
V1_boxcox = ASC_WALK * young + ASC_WALK_NOTYOUNG * (1 - young) + BETA_TIME_WALK * BOXCOX_TIME_1
V2_boxcox = ASC_BIKE * young + ASC_BIKE_NOTYOUNG * (1 - young) + BETA_TIME_BIKE * BOXCOX_TIME_2
V3_boxcox = ASC_PT * young + ASC_PT_NOTYOUNG * (1 - young) + BETA_COST * cost_transit + BETA_TIME_PT * BOXCOX_TIME_3
V4_boxcox = BETA_COST * cost_driving_total + BETA_TIME_DRIVE * BOXCOX_TIME_4

V_boxcox = {1: V1_boxcox, 2: V2_boxcox, 3: V3_boxcox, 4: V4_boxcox}

# Define the model
logprob_3 = models.loglogit(V_boxcox, None, travel_mode)

# Estimate the model
biogeme_3 = bio.BIOGEME(database, logprob_3)
biogeme_3.modelName = 'Model_3'
biogeme_3.generateHtml = False  # Disable HTML file generation
biogeme_3.generatePickle = False  # Disable PICKLE file generation
biogeme_3.save_iterations = False  # Disable ITER file generation
results_model_3 = biogeme_3.estimate()

# Output
print(results_model_3.getEstimatedParameters())



Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


                      Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE          -2.844197      0.279390   -10.180026  0.000000e+00
ASC_BIKE_NOTYOUNG -3.187734      0.254624   -12.519380  0.000000e+00
ASC_PT             1.983684      0.139386    14.231623  0.000000e+00
ASC_PT_NOTYOUNG    1.335467      0.126602    10.548566  0.000000e+00
ASC_WALK           0.118269      0.226521     0.522111  6.015927e-01
ASC_WALK_NOTYOUNG -0.475128      0.228902    -2.075688  3.792277e-02
BETA_COST         -0.147345      0.015345    -9.601969  0.000000e+00
BETA_TIME_BIKE    -3.660758      0.305442   -11.985120  0.000000e+00
BETA_TIME_DRIVE   -3.181831      0.254655   -12.494674  0.000000e+00
BETA_TIME_PT      -2.582341      0.178979   -14.428157  0.000000e+00
BETA_TIME_WALK    -5.533753      0.257560   -21.485321  0.000000e+00
LAMBDA_BOXCOX      0.291986      0.048628     6.004446  1.919860e-09


In [15]:
# Get general statistics for Model 3
general_stats_model_3 = results_model_3.printGeneralStatistics()
print(general_stats_model_3)

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4211.143
Likelihood ratio test for the init. model:	5440.657
Rho-square for the init. model:	0.392
Rho-square-bar for the init. model:	0.391
Akaike Information Criterion:	8446.287
Bayesian Information Criterion:	8524.493
Final gradient norm:	5.6680E-04
Nbr of threads:	8



In [16]:
LR_test = 2 * (results_model_3.data.logLike - results_model_2_0.data.logLike)
print("Log likelihood ratio test:", LR_test)
x_qhi = st.chi2.sf(LR_test, 4)
print("Critical value:", x_qhi)

Log likelihood ratio test: 155.77747567943516
Critical value: 1.1758880085082877e-32


### Model 4

#### Nesting

In [17]:

# Define nest coefficients
MU_MOTOR = Beta('MU_MOTOR', 1, 1, None, 0)  # Nest parameter for motorized transport
MU_PRIVATIZED = Beta('MU_PRIVATIZED', 1, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
nest_motorized = MU_MOTOR, {1: 0,
                         2: 0,
                         3: 1,
                         4: 1}
nest_privatized = MU_PRIVATIZED, {1: 1,
                               2: 1,
                               3: 0,
                               4: 0}

# Combine nests into a list
nests = nest_motorized, nest_privatized

# Define the cross-nested logit model
nested_logit = models.logcnl(V_boxcox, None, nests, travel_mode)

# Estimate the model
biogeme_4_nest = bio.BIOGEME(database, nested_logit)
biogeme_4_nest.modelName = 'Model_4_crossnest'
biogeme_4_nest.generate_html = False  # Disable HTML file generation
biogeme_4_nest.generate_pickle = False  # Disable PICKLE file generation
biogeme_4_nest.save_iterations = False  # Disable ITER file generation
results_model_4 = biogeme_4_nest.estimate()

# Print the estimation results
print(results_model_4.getEstimatedParameters())

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.


                      Value  Active bound  Rob. Std err  Rob. t-test  \
ASC_BIKE          -2.980184           0.0      0.459570    -6.484726   
ASC_BIKE_NOTYOUNG -3.293219           0.0      0.463123    -7.110898   
ASC_PT             1.786318           0.0      0.367604     4.859352   
ASC_PT_NOTYOUNG    1.203011           0.0      0.263227     4.570244   
ASC_WALK          -0.240479           0.0      0.547819    -0.438974   
ASC_WALK_NOTYOUNG -0.806354           0.0      0.501640    -1.607435   
BETA_COST         -0.131397           0.0      0.030816    -4.263984   
BETA_TIME_BIKE    -3.364513           0.0      0.464562    -7.242333   
BETA_TIME_DRIVE   -2.848801           0.0      0.645753    -4.411597   
BETA_TIME_PT      -2.336578           0.0      0.474247    -4.926921   
BETA_TIME_WALK    -5.300348           0.0      0.631268    -8.396352   
LAMBDA_BOXCOX      0.275429           0.0      0.063315     4.350167   
MU_MOTOR           1.115397           0.0      0.209985     5.31

In [18]:
general_stats_model_4 = results_model_4.getGeneralStatistics()
print(results_model_4.printGeneralStatistics())

Number of estimated parameters:	14
Number of free parameters:	13
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4210.831
Likelihood ratio test for the init. model:	5441.281
Rho-square for the init. model:	0.393
Rho-square-bar for the init. model:	0.39
Akaike Information Criterion:	8449.662
Bayesian Information Criterion:	8540.903
Final gradient norm:	2.5636E+01
Nbr of threads:	8



In [19]:
LR_test = 2 * (results_model_4.data.logLike - results_model_3.data.logLike)
print("Log likelihood ratio test:", LR_test)
p_val = st.chi2.sf(LR_test, 2)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 2)
print("Critical value:", x_qhi)


Log likelihood ratio test: 0.6244971812884614
p value: 0.7317995870840155
Critical value: 0.10258658877510109


#### Cross nesting (final model)

In [20]:

# Define nest coefficients
MU_MOTOR = Beta('MU_MOTOR', 3, 1, None, 0)  # Nest parameter for motorized transport
MU_PRIVATIZED = Beta('MU_PRIVATIZED', 3, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
ALPHA_PRIVATIZED = Beta('ALPHA_PRIVATIZED', 0.5, 0, 1, 0)
ALPHA_MOTORIZED = 1 - ALPHA_PRIVATIZED
nest_motorized = MU_MOTOR, {1: 0,
                         2: 0,
                         3: 1,
                         4: ALPHA_MOTORIZED}

nest_privatized = MU_PRIVATIZED, {1: 1,
                               2: 1,
                               3: 0,
                               4: ALPHA_PRIVATIZED}

# Combine nests into a list
nests = nest_motorized, nest_privatized

# Define the cross-nested logit model
crossnested_logit = models.logcnl(V_boxcox, None, nests, travel_mode)

# Estimate the model
biogeme_4_crossnest = bio.BIOGEME(database, crossnested_logit)
biogeme_4_crossnest.modelName = 'Model_4_crossnest'
biogeme_4_crossnest.generate_html = False  # Disable HTML file generation
biogeme_4_crossnest.generate_pickle = False  # Disable PICKLE file generation
biogeme_4_crossnest.save_iterations = False  # Disable ITER file generation
results_model_4_cross = biogeme_4_crossnest.estimate()

# Print the estimation results
print(results_model_4_cross.getEstimatedParameters())

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.


                      Value  Active bound  Rob. Std err  Rob. t-test  \
ALPHA_PRIVATIZED   0.758492           0.0      0.045057    16.834145   
ASC_BIKE          -3.240541           0.0      0.424617    -7.631681   
ASC_BIKE_NOTYOUNG -3.526428           0.0      0.440274    -8.009614   
ASC_PT             1.524780           0.0      0.169408     9.000663   
ASC_PT_NOTYOUNG    1.011705           0.0      0.131528     7.691921   
ASC_WALK          -0.668665           0.0      0.365371    -1.830099   
ASC_WALK_NOTYOUNG -1.208146           0.0      0.409377    -2.951180   
BETA_COST         -0.135564           0.0      0.014319    -9.467580   
BETA_TIME_BIKE    -3.311838           0.0      0.316469   -10.464971   
BETA_TIME_DRIVE   -2.720194           0.0      0.264516   -10.283659   
BETA_TIME_PT      -2.134141           0.0      0.228953    -9.321318   
BETA_TIME_WALK    -5.278732           0.0      0.462927   -11.402958   
LAMBDA_BOXCOX      0.314961           0.0      0.051523     6.11

In [21]:
general_stats_model_4_cross = results_model_4_cross.getGeneralStatistics()
print(results_model_4_cross.printGeneralStatistics())

Number of estimated parameters:	15
Number of free parameters:	14
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-8511.463
Final log likelihood:	-4202.086
Likelihood ratio test for the init. model:	8618.754
Rho-square for the init. model:	0.506
Rho-square-bar for the init. model:	0.505
Akaike Information Criterion:	8434.172
Bayesian Information Criterion:	8531.93
Final gradient norm:	2.1451E+01
Nbr of threads:	8



In [22]:
LR_test = 2 * (results_model_4_cross.data.logLike - results_model_3.data.logLike)
print("Log likelihood ratio test:", LR_test)
p_val = st.chi2.sf(LR_test, 2)
print("p value:", p_val)
x_qhi = st.chi2.ppf(0.05, 2)
print("Critical value:", x_qhi)


Log likelihood ratio test: 18.114515062768987
p value: 0.00011654215191521716
Critical value: 0.10258658877510109


### Market share



#### Computing simulated market share

In [23]:
# size and weight of each strata
strata = {"females_44_less": len(df[(df['age']<=44)&(df['female']==1)]),
         "females_45_more": len(df[(df['age']>=45)&(df['female']==1)]),
         "males_44_less": len(df[(df['age']<=44)&(df['female']==0)]),
         "males_45_more": len(df[(df['age']>=45)&(df['female']==0)])}

total = {"females_44_less": 2841376,
         "females_45_more": 1519948,
         "males_44_less": 2926408,
         "males_45_more": 1379198}

total_population = sum(total.values())
total_sample = sum(strata.values())

weights = {k: total[k] * total_sample / (v * total_population) for k, v in strata.items()}
# k= type of people (female/man and age), v = number of the type k

In [24]:
strata

{'females_44_less': 1623,
 'females_45_more': 965,
 'males_44_less': 1517,
 'males_45_more': 895}

In [25]:
weights

{'females_44_less': 1.0099849525473574,
 'females_45_more': 0.9086698794546592,
 'males_44_less': 1.1128945356168978,
 'males_45_more': 0.889013383029116}

In [26]:
# insert weight as a new column
mask_ = {"females_44_less": (df['age']<=40)&(df['female']==1),
         "females_45_more": (df['age']>=41)&(df['female']==1),
         "males_44_less": (df['age']<=40)&(df['female']==0),
         "males_45_more": (df['age']>=41)&(df['female']==0)}
df['weight'] = 0
for k, v in mask_.items():
    df.loc[v, 'weight'] = weights[k]

C:\Users\Alexander\AppData\Local\Temp\ipykernel_8616\1433649319.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0099849525473574' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[v, 'weight'] = weights[k]


In [27]:
# market share simulated
database = db.Database('LPMC', df)

weight = Variable('weight')
prob_walk = models.cnl(V_boxcox, None, nests, 1)
prob_cycling = models.cnl(V_boxcox, None, nests, 2)
prob_pt = models.cnl(V_boxcox, None, nests, 3)
prob_driving = models.cnl(V_boxcox, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())
simulated_values

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


,weight,prob.walk,prob.cycling,prob.pt,prob.driving
0,0.908670,0.012469,0.021336,0.290703,0.675492
1,0.908670,0.705114,0.023210,0.084704,0.186973
2,0.889013,0.000750,0.014409,0.371268,0.613574
3,0.889013,0.000098,0.011199,0.756970,0.231734
4,0.889013,0.000404,0.010728,0.498317,0.490551
...,...,...,...,...,...
4995,0.889013,0.002698,0.011996,0.425209,0.560096
4996,1.112895,0.000128,0.004693,0.058251,0.936928
4997,1.112895,0.105707,0.053616,0.148915,0.691761
4998,1.112895,0.023334,0.055601,0.430085,0.490980


In [28]:
simulated_values['weighted walk'] = simulated_values['weight'] * simulated_values['prob.walk']
simulated_values['weighted cycling'] = simulated_values['weight'] * simulated_values['prob.cycling']
simulated_values['weighted pt'] = simulated_values['weight'] * simulated_values['prob.pt']
simulated_values['weighted driving'] = simulated_values['weight'] * simulated_values['prob.driving']

In [29]:
market_share_walk = simulated_values['weighted walk'].mean()
market_share_cycling = simulated_values['weighted cycling'].mean()
market_share_pt = simulated_values['weighted pt'].mean()
market_share_driving = simulated_values['weighted driving'].mean()

print(f"Market share of walk (simulated): {100*market_share_walk:.1f}%")
print(f"Market share of cycling(simulated): {100*market_share_cycling:.1f}%")
print(f"Market share of pt(simulated): {100*market_share_pt:.1f}%")
print(f"Market share of driving(simulated): {100*market_share_driving:.1f}%")

Market share of walk (simulated): 17.8%
Market share of cycling(simulated): 2.9%
Market share of pt(simulated): 35.2%
Market share of driving(simulated): 43.0%


In [30]:
help(simulate)

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Built-in subclasses:
 |      StgDict
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>va

#### Computing actual market share

In [31]:
#actual market share:

# weighted market shares using actual choices
mask_choice = {"females_44_less":{'walk': len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']<=44)&(df['female']==1)&(df['travel_mode']==4)])},
              "females_45_more": {'walk': len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']>=45)&(df['female']==1)&(df['travel_mode']==4)])},
              "males_44_less": {'walk': len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']<=44)&(df['female']==0)&(df['travel_mode']==4)])},
              "males_45_more": {'walk': len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==1)]),
                                  'cycling':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==2)]),
                                  'pt':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==3)]),
                                  'driving':len(df[(df['age']>=45)&(df['female']==0)&(df['travel_mode']==4)])}}

market_share_walk_weighted = sum([weights[k] * v['walk'] for k, v in mask_choice.items()])/total_sample
market_share_cycling_weighted = sum([weights[k] * v['cycling'] for k, v in mask_choice.items()])/total_sample
market_share_pt_weighted = sum([weights[k] * v['pt'] for k, v in mask_choice.items()])/total_sample
market_share_driving_weighted = sum([weights[k] * v['driving'] for k, v in mask_choice.items()])/total_sample

In [32]:
print(f"Weighted market share of walk: {100*market_share_walk_weighted:.1f}%")
print(f"Weighted market share of cycling: {100*market_share_cycling_weighted:.1f}%")
print(f"Weighted market share of pt: {100*market_share_pt_weighted:.1f}%")
print(f"Weighted market share of driving: {100*market_share_driving_weighted:.1f}%")

Weighted market share of walk: 18.1%
Weighted market share of cycling: 3.0%
Weighted market share of pt: 35.5%
Weighted market share of driving: 43.4%


#### Confidence interval

**Microsimulation Process:**

1. **Model Estimation:**
   - A choice model (like a multinomial logit model) is estimated using observed data.
   - The model estimates the probability $ P_n(i \mid x_n; \hat{\theta}) $ that individual $ n $ chooses alternative $ i $ based on their attributes $ x_n $ and the estimated parameters $ \hat{\theta} $.

2. **Simulation of Choices:**
   - For each individual in the sample, the choice model is used to simulate choices.
   - This typically involves drawing a random number and comparing it to the cumulative probability distribution of the choices to determine which alternative is selected.
   - Each simulation is repeated $ R $ times to capture the variability of choices due to the randomness in the model.

3. **Aggregation:**
   - The number of times each alternative is chosen across all simulations is counted to calculate the simulated number of individuals $ \hat{N}(i) $ choosing each alternative.
   - Aggregate market shares are then estimated by averaging over all simulations.

**Calculation of Aggregate Market Shares:**

1. **Number of Individuals Choosing Alternative $ i $:**
   - This is calculated as the average number of times alternative $ i $ is chosen across all $ R $ simulations for each individual.
   $$ \hat{N}(i) = \frac{1}{R} \sum_{n=1}^{N} \sum_{r=1}^{R} \hat{y}_{inr} $$

2. **Share of the Population Choosing Alternative $ i $:**
   - This is the proportion of the population that is estimated to choose alternative $ i $, averaged over all simulations.
   $$ \hat{W}(i) = \frac{1}{NR} \sum_{n=1}^{N} \sum_{r=1}^{R} \hat{y}_{inr} $$

**Calculation of Confidence Intervals:**

To calculate confidence intervals for the simulated market shares:

1. **Bootstrap Method:**
   - The bootstrap method involves resampling the simulated choice data with replacement and recalculating $ \hat{N}(i) $ and $ \hat{W}(i) $ for each resample.
   - This process is repeated many times (e.g., 1000 or more) to create an empirical distribution of the market shares.

2. **Confidence Interval Estimation:**
   - Confidence intervals are then derived from the empirical distribution of the bootstrapped market shares.
   - For example, the 95% confidence interval can be estimated using the 2.5th and 97.5th percentiles of the bootstrapped market shares.


In [33]:

# Number of bootstrap samples
n_bootstraps = 1000
confidence_level = 0.95

# Initialize an array to store the bootstrap market share estimates
bootstrap_market_shares = {
    'walk': [],
    'cycling': [],
    'pt': [],
    'driving': []
}

# Perform bootstrapping
for i in range(n_bootstraps):
    # Sample with replacement from the simulated_values
    sample = simulated_values.sample(n=len(simulated_values), replace=True)
    
    # Calculate weighted market shares for the bootstrap sample
    for mode in ['walk', 'cycling', 'pt', 'driving']:
        market_share = (sample['weight'] * sample[f'prob.{mode}']).mean()
        bootstrap_market_shares[mode].append(market_share)

# Calculate the confidence intervals
lower_bound = (1 - confidence_level) / 2
upper_bound = 1 - lower_bound

market_share_confidence_intervals = {}
for mode in ['walk', 'cycling', 'pt', 'driving']:
    lower = np.percentile(bootstrap_market_shares[mode], lower_bound * 100)
    upper = np.percentile(bootstrap_market_shares[mode], upper_bound * 100)
    market_share_confidence_intervals[mode] = (lower, upper)

# Print the confidence intervals
for mode, interval in market_share_confidence_intervals.items():
    print(f"{mode.capitalize()} market share 95% confidence interval: {100*interval[0]:.1f}% - {100*interval[1]:.1f}%")


Walk market share 95% confidence interval: 17.1% - 18.4%
Cycling market share 95% confidence interval: 2.8% - 2.9%
Pt market share 95% confidence interval: 34.5% - 35.8%
Driving market share 95% confidence interval: 42.4% - 43.6%


### Forecasting

We consider two scenarios:
1. An increase of 1.50 GBP for car users
2. A decrease of the public transport charge of 20%


#### Predicted market share in both cases:

In [34]:
#predicted market share in case of increase of car costs:

V4_s1 = (BETA_COST * (cost_driving_total + 1.5) + BETA_TIME_DRIVE * BOXCOX_TIME_4)
V_s1 = {1: V1_boxcox, 2: V2_boxcox, 3: V3_boxcox, 4: V4_s1}
prob_walk = models.cnl(V_s1, None, nests, 1)
prob_cycling = models.cnl(V_s1, None, nests, 2)
prob_pt = models.cnl(V_s1, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s1, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving_scenario1
}

biosim_s1 = bio.BIOGEME(database, simulate) #using database defined in market share
simulated_values_s1 = biosim_s1.simulate(results_model_4_cross.getBetaValues())

simulated_values_s1['weighted walk'] = simulated_values_s1['weight'] * simulated_values_s1['prob.walk']
simulated_values_s1['weighted cycling'] = simulated_values_s1['weight'] * simulated_values_s1['prob.cycling']
simulated_values_s1['weighted pt'] = simulated_values_s1['weight'] * simulated_values_s1['prob.pt']
simulated_values_s1['weighted driving'] = simulated_values_s1['weight'] * simulated_values_s1['prob.driving']

market_share_walk_s1 = simulated_values_s1['weighted walk'].mean()
market_share_cycling_s1 = simulated_values_s1['weighted cycling'].mean()
market_share_pt_s1 = simulated_values_s1['weighted pt'].mean()
market_share_driving_s1 = simulated_values_s1['weighted driving'].mean()

print('Scenario 1: increase car cost by 1.5 pounds')
print(f"Market share of walk: {100*market_share_walk_s1:.2f}%")
print(f"Market share of cycling: {100*market_share_cycling_s1:.2f}%")
print(f"Market share of pt: {100*market_share_pt_s1:.2f}%")
print(f"Market share of driving: {100*market_share_driving_s1:.2f}%")

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Scenario 1: increase car cost by 1.5 pounds
Market share of walk: 18.78%
Market share of cycling: 3.15%
Market share of pt: 38.68%
Market share of driving: 38.18%


In [35]:
#predicted market share in case of decrease in public transport

V3_s2 = ASC_PT * young + ASC_PT_NOTYOUNG * (1 - young) + BETA_COST * cost_transit * 0.8 + BETA_TIME_PT * BOXCOX_TIME_3
V_s2 = {1: V1_boxcox, 2: V2_boxcox, 3: V3_s2, 4: V4_boxcox}
prob_walk = models.cnl(V_s2, None, nests, 1)
prob_cycling = models.cnl(V_s2, None, nests, 2)
prob_pt = models.cnl(V_s2, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s2, None, nests, 4)
simulate = {
    'weight': weight,
    'prob.walk': prob_walk,
    'prob.cycling': prob_cycling,
    'prob.pt': prob_pt,
    'prob.driving': prob_driving_scenario1
}

biosim_s2 = bio.BIOGEME(database, simulate) #using database defined in market share
simulated_values_s2 = biosim_s2.simulate(results_model_4_cross.getBetaValues())

simulated_values_s2['weighted walk'] = simulated_values_s2['weight'] * simulated_values_s2['prob.walk']
simulated_values_s2['weighted cycling'] = simulated_values_s2['weight'] * simulated_values_s2['prob.cycling']
simulated_values_s2['weighted pt'] = simulated_values_s2['weight'] * simulated_values_s2['prob.pt']
simulated_values_s2['weighted driving'] = simulated_values_s2['weight'] * simulated_values_s2['prob.driving']

market_share_walk_s2 = simulated_values_s2['weighted walk'].mean()
market_share_cycling_s2 = simulated_values_s2['weighted cycling'].mean()
market_share_pt_s2 = simulated_values_s2['weighted pt'].mean()
market_share_driving_s2 = simulated_values_s2['weighted driving'].mean()

print('Scenario 2: decrease public transport costs by 20%')
print(f"Market share of walk: {100*market_share_walk_s2:.2f}%")
print(f"Market share of cycling: {100*market_share_cycling_s2:.2f}%")
print(f"Market share of pt: {100*market_share_pt_s2:.2f}%")
print(f"Market share of driving: {100*market_share_driving_s2:.2f}%")

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Scenario 2: decrease public transport costs by 20%
Market share of walk: 17.71%
Market share of cycling: 2.85%
Market share of pt: 36.05%
Market share of driving: 42.20%


When wanting to decrease the share of car, we should consider the first scenario, as the simulated market share of driving is **39.13%**. In the second scenario, the car market share is **39.35%**, thus higher.

### Highest pt revenue 

We want to check in which scenario the public transportation revenue is the highest. To do so, we need to compute the revenue in all 3 cases (no changes, increase in car costs, decrease in pt costs).

In [36]:
database = db.Database('LPMC', df)
weight = Variable('weight')

In [37]:
#no change in policy

prob_walk = models.cnl(V_boxcox, None, nests, 1)
prob_cycling = models.cnl(V_boxcox, None, nests, 2)
prob_pt = models.cnl(V_boxcox, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_boxcox, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (no changes in policy): {simulated_values['revenues PT'].sum()}")


It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (no changes in policy): 3347.2600838413987


In [38]:
#scenario 1 (increase in car costs)

prob_walk = models.cnl(V_s1, None, nests, 1)
prob_cycling = models.cnl(V_s1, None, nests, 2)
prob_pt = models.cnl(V_s1, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s1, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (scenario 1 policy): {simulated_values['revenues PT'].sum()}")

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (scenario 1 policy): 3638.911952357479


In [49]:
#scenario 2 (decrease in pt costs)

prob_walk = models.cnl(V_s2, None, nests, 1)
prob_cycling = models.cnl(V_s2, None, nests, 2)
prob_pt = models.cnl(V_s2, None, nests, 3)
prob_driving_scenario1 = models.cnl(V_s2, None, nests, 4)

simulate = {
    'weight': weight,
    'revenues PT': prob_pt * cost_transit * 0.8

}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

print(f"Public transport revenues (scenario 2 policy): {simulated_values['revenues PT'].sum()}")

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


Public transport revenues (scenario 2 policy): 2783.606794491914


The **second scenario** gives the highest revenues for public transport.

### Average value of time

We want to compute the average VOT for both car and public transportation (in GBP/hour). To do so, we use the following formula:

$$ \text{(VOT)}_\text{i} = \frac{\partial  \text{Utility}_i / \partial \text{duration}_i}{\partial  \text{Utility}_i / \partial \text{cost}_i} , i \in \{car,pt\} $$


In [40]:
V_pt = models.cnl(V_boxcox, None, nests, 3)
V_driving = models.cnl(V_boxcox, None, nests, 4)

#idk why it doesn't work without defining both of the utilities upper ????

vot_pt = Derive(V_pt, 'dur_pt_total') / Derive(V_pt, 'cost_transit')
vot_car = Derive(V_driving, 'dur_driving') / Derive(V_driving, 'cost_driving_total')

simulate = {
    'weight': weight,
    'WTP PT time': vot_pt,
    'WTP CAR time': vot_car,
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())



It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


In [41]:
print(f"Average value of time for public transport: {(simulated_values['weight']*simulated_values['WTP PT time']).mean()}  GBP/hour")
print(f"Average value of time for car: {(simulated_values['weight']*simulated_values['WTP CAR time']).mean()} GBP/hour")

Average value of time for public transport: 34.8007569651067  GBP/hour
Average value of time for car: 71.577470284145 GBP/hour


### Direct and cross aggregate elasticities

Now we need to compute the direct and cross elasticites of car costs and public transport costs. 

The **direct price elasticity** for the car is the percent change in pt change resulting from a 1% change in car costs. The formula is given by:

$$ E^{car}_{pt} =  \frac{(cost_{transit})}{(cost_{driving_total})} \cdot \frac{\partial  (cost_{driving_total})}{\partial (cost_{transit})}$$

The **cross price elasticity** is given by the following formula:


In [58]:
prob_pt = models.cnl(V_boxcox, None, nests, 3)
prob_driving = models.cnl(V_boxcox, None, nests, 4)

#direct elasticities 
direct_elas_pt_cost = Derive(prob_pt, 'cost_transit') * cost_transit / prob_pt
direct_elas_driving_cost = Derive(prob_driving, 'cost_driving_total') * cost_driving_total / prob_driving

simulate = {
    'weight': weight,
    'prob.driving': prob_driving,
    'prob.pt': prob_pt,
    'direct_elas_pt_cost': direct_elas_pt_cost,
    'direct_elas_driving_cost': direct_elas_driving_cost
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
It is recommended to define the nests of the cross-nested logit model using the objects OneNestForNestedLogit and NestsForCrossNestedLogit defined in biogeme.nests.
The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


In [59]:
simulated_values['numerator_pt_cost'] = simulated_values['weight'] * simulated_values['prob.pt'] * simulated_values['direct_elas_pt_cost']
simulated_values['numerator_driving_cost'] = simulated_values['weight'] * simulated_values['prob.driving'] * simulated_values['direct_elas_driving_cost']
simulated_values['denominator_pt_cost'] = simulated_values['weight'] * simulated_values['prob.pt']
simulated_values['denominator_driving_cost'] = simulated_values['weight'] * simulated_values['prob.driving']

In [60]:
#aggregate elasticities

agg_elast_pt_cost = simulated_values['numerator_pt_cost'].sum()/simulated_values['denominator_pt_cost'].sum()
agg_elast_driving_cost = simulated_values['numerator_driving_cost'].sum()/simulated_values['denominator_driving_cost'].sum()

print(f"Elasticity of public transport cost: {agg_elast_pt_cost}")
print(f"Elasticity of public driving cost: {agg_elast_driving_cost}")

Elasticity of public transport cost: -0.1276779313129734
Elasticity of public driving cost: -0.08335709271075455


#### Cross elastisities

In [53]:
# Compute the derivatives for cross elasticity
cross_elas_pt_driving_cost = Derive(prob_pt, 'cost_driving_total') * cost_driving_total / prob_pt
cross_elas_driving_pt_cost = Derive(prob_driving, 'cost_transit') * cost_transit / prob_driving

# Update the simulation dictionary

simulate = {
    'weight': weight,
    'prob.driving': prob_driving,
    'prob.pt': prob_pt,
    'cross_elas_pt_driving_cost': cross_elas_pt_driving_cost,
    'cross_elas_driving_pt_cost': cross_elas_driving_pt_cost
}

# Rerun the simulation with updated dictionary
biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model_4_cross.getBetaValues())

# Compute numerators and denominators for cross elasticities
simulated_values['numerator_pt_driving_cost'] = simulated_values['weight'] * simulated_values['prob.pt'] * simulated_values['cross_elas_pt_driving_cost']
simulated_values['numerator_driving_pt_cost'] = simulated_values['weight'] * simulated_values['prob.driving'] * simulated_values['cross_elas_driving_pt_cost']
simulated_values['denominator_pt_cost'] = simulated_values['weight'] * simulated_values['prob.pt']
simulated_values['denominator_driving_cost'] = simulated_values['weight'] * simulated_values['prob.driving']

# Aggregate cross elasticities
agg_cross_elast_pt_driving_cost = simulated_values['numerator_pt_driving_cost'].sum() / simulated_values['denominator_pt_cost'].sum()
agg_cross_elast_driving_pt_cost = simulated_values['numerator_driving_pt_cost'].sum() / simulated_values['denominator_driving_cost'].sum()

# Print the results
print(f"Cross elasticity of public transport with respect to driving cost: {agg_cross_elast_pt_driving_cost}")
print(f"Cross elasticity of driving with respect to public transport cost: {agg_cross_elast_driving_pt_cost}")


The sum of the weights (4939.940013322009) is different from the sample size (5000). Multiply the weights by 1.0121580396757899 to reconcile the two.


KeyError: 'denominator_pt_cost'